### portmy database: stocks table
### stock database: buy, price tables

In [17]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 9)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-09
yesterday: 2023-07-07 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 7)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2023-06-30
yesterday: 2023-07-07


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,BBL,2023-07-07,155.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'BBL'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,50,BBL,SET50 / SETCLMV / SETHD / SETTHSI,161.00,163.00,124.50,10.95,0.60,"19,088.43","307,323.71","2,107.11",0.86,50,2017-07-23 07:24:26.222243,2023-01-31 03:07:59.829483


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "BBL" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,BBL,2023-07-07,156.00,157.00,155.00,"4,881,835",155.00
1,BBL,2023-07-06,156.00,158.00,156.00,"13,692,556",158.00
2,BBL,2023-07-05,159.00,161.00,158.00,"10,012,829",161.00
3,BBL,2023-07-04,160.50,161.50,159.50,"9,294,555",159.50
4,BBL,2023-07-03,159.50,160.50,158.00,"8,286,583",160.00


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2023-07-07' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART')
ORDER BY name, date


30

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2023-06-30' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BBL', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART') 
ORDER BY name


30

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(30, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,AH,35.75,34.75,2.88
1,ASK,25.00,25.00,0.00
2,ASP,2.80,2.78,0.72
3,AWC,4.60,4.94,-6.88
4,BBL,156.00,159.50,-2.19
5,BCH,17.40,17.70,-1.69
6,CPNREIT,11.00,10.80,1.85
7,DIF,10.90,11.10,-1.80
8,GVREIT,8.25,8.35,-1.20
9,ICHI,13.60,13.80,-1.45


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
15,KTC,46.00,49.50,-7.07%,Worse
3,AWC,4.60,4.94,-6.88%,Worse
14,KCE,36.50,39.00,-6.41%,Worse
18,ORI,10.10,10.60,-4.72%,Worse
13,JMT,36.25,37.50,-3.33%,Worse
10,IVL,33.25,34.25,-2.92%,Worse
4,BBL,156.00,159.50,-2.19%,Worse
7,DIF,10.90,11.10,-1.80%,Worse
5,BCH,17.40,17.70,-1.69%,Worse
9,ICHI,13.60,13.80,-1.45%,Worse


In [20]:
file_name = 'trend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(data_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [21]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
3,AWC,4.60,4.94,-6.88,Worse
4,BBL,156.00,159.50,-2.19,Worse
5,BCH,17.40,17.70,-1.69,Worse
7,DIF,10.90,11.10,-1.80,Worse
8,GVREIT,8.25,8.35,-1.20,Worse
9,ICHI,13.60,13.80,-1.45,Worse
10,IVL,33.25,34.25,-2.92,Worse
13,JMT,36.25,37.50,-3.33,Worse
14,KCE,36.50,39.00,-6.41,Worse
15,KTC,46.00,49.50,-7.07,Worse


In [22]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,46.67%
Better,40.00%
No Change,13.33%


In [23]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(251, 8)

In [24]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [25]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(30, 13)

In [26]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
15,KTC,49.50,46.00,-7.07%,Worse,SET50
3,AWC,4.94,4.60,-6.88%,Worse,SET50
13,JMT,37.50,36.25,-3.33%,Worse,SET50
10,IVL,34.25,33.25,-2.92%,Worse,SET50
4,BBL,159.50,156.00,-2.19%,Worse,SET50
21,SCC,322.00,322.00,0.00%,No Change,SET50
12,JMART,15.90,16.30,2.52%,Better,SET50
19,PTTGC,37.25,38.50,3.36%,Better,SET50


In [27]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,62.50%
Better,25.00%
No Change,12.50%


In [28]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
14,KCE,39.00,36.50,-6.41%,Worse,SET100
18,ORI,10.60,10.10,-4.72%,Worse,SET100
5,BCH,17.70,17.40,-1.69%,Worse,SET100
24,STA,17.00,17.20,1.18%,Better,SET100
20,RCL,23.70,24.00,1.27%,Better,SET100
23,SINGER,9.00,9.15,1.67%,Better,SET100


In [29]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,50.00%
Better,50.00%


In [30]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
7,DIF,11.10,10.90,-1.80%,Worse,SET999
9,ICHI,13.80,13.60,-1.45%,Worse,SET999
8,GVREIT,8.35,8.25,-1.20%,Worse,SET999
29,WHART,10.60,10.50,-0.94%,Worse,SET999
17,NER,4.54,4.50,-0.88%,Worse,SET999
28,WHAIR,7.10,7.05,-0.70%,Worse,SET999
1,ASK,25.00,25.00,0.00%,No Change,SET999
11,JASIF,7.05,7.05,0.00%,No Change,SET999
22,SENA,2.98,2.98,0.00%,No Change,SET999
26,TFFIF,7.55,7.60,0.66%,Better,SET999


In [31]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,43.75%
Worse,37.50%
No Change,18.75%


In [32]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [33]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
